In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime

In [94]:
yf.Ticker("MSFT").info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 221000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related Client Access Licenses; GitHub that provides a collaboration platform and code hosting service for developers; Nuance provides healthcare and enterprise AI solutions; and Azure, a cloud platform. It also offers enterprise

In [52]:
df = yf.Ticker("MSFT").cashflow

In [88]:
df.columns[0].year

2022

In [27]:
def grahams_calc(ticker_symbol, growth_rate):
    stock = yf.Ticker(ticker_symbol)
    EPS = stock.info['trailingEps']
    PE_no_growth = 7
    growth_rate = growth_rate #10 #stock growth rate %
    ave_AAA_yield = 4.4 #%
    curr_AAA_yield = 2.57 #(get online)
    value = ( EPS * (PE_no_growth + growth_rate) * ave_AAA_yield ) /  curr_AAA_yield
    
    return value




In [143]:
grahams_calc("LMT", 6)

485.8661478599222

In [134]:
def IVcalc(ticker_symbol, est_g_05, est_g_510, est_term_mult):
    
    curr_year = datetime.today().year
    
    #pull current ir
    IR = 0.0400

    stock = yf.Ticker(ticker_symbol)
    
    shares_outstanding = stock.info['sharesOutstanding']
    
    #Get cashflow from most recent year
    stock_cf = stock.cashflow
    
    #index 0 gets us the most recent dates
    year = stock_cf.columns[0].year
    print(year)
    cash_from_op = stock_cf.loc['Total Cash From Operating Activities'].iloc[0]
    cap_ex = stock_cf.loc['Capital Expenditures'].iloc[0]
    cash_flow = cash_from_op + cap_ex #cash from ops - capex (but is a + as -ve vals used in bal sheet)
    if (cash_flow < 0):
        print("Negative cash flow for current year, please take note. [" + str(cash_flow) + "]")
    
    
    estimated_growth = [est_g_05, est_g_510]
    
    terminal_multiple = est_term_mult
    
    #Estimate for next 10 years
    est_cash_flows = []
    for year in range(1,11):
        choice = 0
        if year > 5:
            choice = 1
        est_cf = cash_flow * ((1 + estimated_growth[choice])** year)
        est_cash_flows.append(est_cf)
    
    
    terminal_value = terminal_multiple * est_cash_flows[-2]
    
    present_val_cf = []
    
    #RFROR
    #WACC #temp put 4.5%
    discount_rate = 0.045
    
    for year in range(10):
        net_pres = est_cash_flows[year] / ((1 + discount_rate) ** (year + 1))
        present_val_cf.append(net_pres)
    
    NPV = sum(present_val_cf) + terminal_value
    
    IV = NPV / shares_outstanding
    
    
    return IV

In [137]:
IVcalc("LMT", 0.07,0.03, 5)

2021


480.2928776616334

In [140]:
def IVCalcAve(ticker_symbol,best_p, worst_p, ave_p, cases = [[0.2,0.1,15],[0.05,0.02,5],[0.1,0.05,10]]):
    stock = yf.Ticker(ticker_symbol)
    financial_currency = stock.info['financialCurrency']
    
    #If there is an input
    best_case = cases[0]
    worst_case = cases[1]
    average_case = cases[2]
    
    #probabilities
    probs = [best_p, worst_p, ave_p]
    
    cases = [best_case, worst_case, average_case]
    print(cases)
    expected_IV = 0
    for i in range(len(cases)):
        case = cases[i]
        IV = IVcalc(ticker_symbol, case[0], case[1], case[2]) * probs[i]
        expected_IV += IV
    return ticker_symbol + ":" + str(int(expected_IV)) + financial_currency
    

In [141]:
IVCalcAve("LMT", 0.2,0.3,0.5, [[0.08,0.04,5],[0.03,0.01,4],[0.06,0.03, 5]])

[[0.08, 0.04, 5], [0.03, 0.01, 4], [0.06, 0.03, 5]]
2021
2021
2021


'LMT:454USD'